In [1]:
pip install lxml pillow Cython contextlib2 tf_slim



  Using cached lxml-5.2.2-cp310-cp310-win_amd64.whl (3.8 MB)
  Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl (2.6 MB)
     ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
      --------------------------------------- 0.0/2.8 MB 1.9 MB/s eta 0:00:02
      --------------------------------------- 0.1/2.8 MB 1.1 MB/s eta 0:00:03
      --------------------------------------- 0.1/2.8 MB 1.1 MB/s eta 0:00:03
      --------------------------------------- 0.1/2.8 MB 1.1 MB/s eta 0:00:03
      --------------------------------------- 0.1/2.8 MB 1.1 MB/s eta 0:00:03
      --------------------------------------- 0.1/2.8 MB 1.1 MB/s eta 0:00:03
     -- ------------------------------------- 0.2/2.8 MB 458.0 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/2.8 MB 458.0 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/2.8 MB 458.0 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/2.8 MB 458.0 kB/s eta 0:00:06
     -- -------


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install object_detection

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.5 MB 1.9 MB/s eta 0:00:01
     - -------------------------------------- 0.1/1.5 MB 787.7 kB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.5 MB 581.0 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 653.6 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'e:\\ai projects\\verona_mites\\bee-image-classifier-master\\bee-image-classifier-master\\tf_env\\lib\\site-packages\\numpy\\linalg\\_umath_linalg.cp310-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip uninstall numpy pandas
!pip install numpy pandas


^C



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':
        return 1
    else:
        None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [1]:
! pip install tensorflow==2.8.0  # Adjust the version as needed
! pip install pandas lxml pillow
! pip install tensorflow-object-detection-api


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pandas-2.2.2-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached lxml-5.2.2-cp310-cp310-win_amd64.whl (3.8 MB)
  Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached numpy-2.0.1-cp310-cp310-win_amd64.whl (16.6 MB)



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/577.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/577.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/577.4 kB ? eta -:--:--
     - ----------------------------------- 20.5/577.4 kB 640.0 kB/s eta 0:00:01
     - ----------------------------------- 30.7/577.4 kB 435.7 kB/s eta 0:00:02
     --- --------------------------------- 61.4/577.4 kB 465.5 kB/s eta 0:00:02
     --- --------------------------------- 61.4/577.4 kB 465.5 kB/s eta 0:00:02
     --- --------------------------------- 61.4/577.4 kB 465.5 kB/s eta 0:00:02
     ----- ------------------------------- 92.2/577.4 kB 327.7 kB/s eta 0:00:02
     ----- ------------------------------- 92.2/577.4 kB 327.7 kB/s eta 0:00:02
     ------- ---------------------------- 122.9/577.4 kB 343.4 kB/s eta 0:00:02
     ------- ---------------------------- 122.9/577.4 kB 343.4 kB/s eta 0:00:02
     -------- --------------------------- 143.4/577.4 kB 327.

  DEPRECATION: tensorflow-object-detection-api is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.17.0


In [3]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()

if __name__ == '__main__':
    tf.compat.v1.app.run()


InvalidArgumentError: NewRandomAccessFile failed to Create/Open: dataset/images\0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option

In [4]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()

if __name__ == '__main__':
    tf.compat.v1.app.run()


InvalidArgumentError: NewRandomAccessFile failed to Create/Open: dataset/images\0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option

In [6]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    # with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    #     encoded_jpg = fid.read()
    # encoded_jpg_io = io.BytesIO(encoded_jpg)
    # image = Image.open(encoded_jpg_io)
    # width, height = image.size
    # filename = group.filename
    filename = group.filename.item()
    full_image_path = os.path.join(path, filename)  # Use os.path.join

    if not os.path.exists(full_image_path):
        print(f"Warning: Image not found: {full_image_path}")
        return None  # Skip this example if image is missing

    with open(full_image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# def main(_):
#     writer = tf.io.TFRecordWriter('train.record')
#     path = os.path.join(IMAGE_PATH)
#     examples = xml_to_csv(ANNOTATION_PATH)
#     grouped = examples.groupby('filename')
#     for filename, group in grouped:
#         tf_example = create_tf_example(group, path)
#         writer.write(tf_example.SerializeToString())
#     writer.close()

# if __name__ == '__main__':
#     tf.compat.v1.app.run()
def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = IMAGE_PATH  # Use the corrected path directly
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        tf_example = create_tf_example(group, path)
        if tf_example is not None:  # Skip if image is missing
            writer.write(tf_example.SerializeToString())
    writer.close()

if __name__ == '__main__':
    tf.compat.v1.app.run()

ValueError: can only convert an array of size 1 to a Python scalar

In [21]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        print(f"Path: {os.path.join(path, filename)}")
        if not os.path.exists(os.path.join(path, filename)):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()

# Call the main function directly
if __name__ == '__main__':
    main(_)


Processing file: IMG_5536.jpg
Path: dataset/images\IMG_5536.jpg


InvalidArgumentError: NewRandomAccessFile failed to Create/Open: dataset/images\0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option

In [25]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(os.path.join(path, '*.xml')):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        raise ValueError(f"Unknown class label: {row_label}")

def create_tf_example(group, path):
    filename = group['filename'].iloc[0]
    file_path = os.path.join(path, filename)
    try:
        with tf.io.gfile.GFile(file_path, 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
    except Exception as e:
        print(f"Error reading image file {filename}: {e}")
        return None

    filename_encoded = filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        try:
            classes.append(class_text_to_int(row['class']))
        except ValueError as e:
            print(f"Skipping annotation for {filename} due to error: {e}")
            return None

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename_encoded),
        'image/source_id': dataset_util.bytes_feature(filename_encoded),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main():
    writer = tf.io.TFRecordWriter('train.record')
    path = IMAGE_PATH
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        if not os.path.exists(os.path.join(path, filename)):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
        else:
            print(f"Skipped file {filename} due to read error or unknown class label.")
    writer.close()

# Call the main function directly
if __name__ == '__main__':
    main()

Processing file: IMG_5536.jpg
Skipping annotation for IMG_5536.jpg due to error: Unknown class label: Varroa
Skipped file IMG_5536.jpg due to read error or unknown class label.
Processing file: IMG_5559.jpg
Skipping annotation for IMG_5559.jpg due to error: Unknown class label: Varroa
Skipped file IMG_5559.jpg due to read error or unknown class label.
Processing file: IMG_5560.jpg
Skipping annotation for IMG_5560.jpg due to error: Unknown class label: Varroa
Skipped file IMG_5560.jpg due to read error or unknown class label.
Processing file: IMG_5561.jpg
Skipping annotation for IMG_5561.jpg due to error: Unknown class label: Varroa
Skipped file IMG_5561.jpg due to read error or unknown class label.
Processing file: IMG_5562.jpg
Skipping annotation for IMG_5562.jpg due to error: Unknown class label: Varroa
Skipped file IMG_5562.jpg due to read error or unknown class label.
Processing file: IMG_5582.jpg
Skipping annotation for IMG_5582.jpg due to error: Unknown class label: Varroa
Skippe

In [24]:
path = os.path.join(IMAGE_PATH, filename.replace('/', '\\'))

full_image_path = os.path.join(path, '{}'.format(group.filename))
print(f"Opening image: {full_image_path}")


NameError: name 'filename' is not defined

In [10]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images/'
ANNOTATION_PATH = 'dataset/annotations/'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        print(f"Path: {os.path.join(path, filename)}")
        if not os.path.exists(os.path.join(path, filename)):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()

if __name__ == '__main__':
    tf.compat.v1.app.run()


Processing file: IMG_5536.jpg
Path: dataset/images/IMG_5536.jpg


InvalidArgumentError: NewRandomAccessFile failed to Create/Open: dataset/images/0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option

In [11]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    try:
        with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
    except Exception as e:
        print(f"Error reading image file {group.filename}: {e}")
        return None

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        print(f"Path: {os.path.join(path, filename)}")
        if not os.path.exists(os.path.join(path, filename)):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
        else:
            print(f"Skipped file {filename} due to read error.")
    writer.close()

if __name__ == '__main__':
    tf.compat.v1.app.run()


Processing file: IMG_5536.jpg
Path: dataset/images\IMG_5536.jpg
Error reading image file 0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object: NewRandomAccessFile failed to Create/Open: dataset/images\0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option
Skipped file IMG_5536.jpg due to read error.
Processing file: IMG_5559.jpg
Path: dataset/images\IMG_5559.jpg
Error reading image file 9     IMG_5559.jpg
10    IMG_5559.jpg
11    IMG_5559.jpg
Name: filename, dtype: object: NewRandomAccessFile failed to Create/Open: dataset/images\9     IMG_5559.jpg
10    IMG_5559.jpg
11    IMG_5559.jpg
Name: filename, dtype: object : The filen

SystemExit: 

e:\AI Projects\Verona_Mites\bee-image-classifier-master\bee-image-classifier-master\tf_new\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'object_class_name':  # Change this to your object class name
        return 1
    else:
        None

def create_tf_example(group, path):
    try:
        with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
    except Exception as e:
        print(f"Error reading image file {group.filename}: {e}")
        return None

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main():
    writer = tf.io.TFRecordWriter('train.record')
    path = os.path.join(IMAGE_PATH)
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        print(f"Path: {os.path.join(path, filename)}")
        if not os.path.exists(os.path.join(path, filename)):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
        else:
            print(f"Skipped file {filename} due to read error.")
    writer.close()

# Call the main function directly
if __name__ == '__main__':
    main()


Processing file: IMG_5536.jpg
Path: dataset/images\IMG_5536.jpg
Error reading image file 0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object: NewRandomAccessFile failed to Create/Open: dataset/images\0    IMG_5536.jpg
1    IMG_5536.jpg
2    IMG_5536.jpg
3    IMG_5536.jpg
4    IMG_5536.jpg
5    IMG_5536.jpg
6    IMG_5536.jpg
7    IMG_5536.jpg
8    IMG_5536.jpg
Name: filename, dtype: object : The filename, directory name, or volume label syntax is incorrect.
; no protocol option
Skipped file IMG_5536.jpg due to read error.
Processing file: IMG_5559.jpg
Path: dataset/images\IMG_5559.jpg
Error reading image file 9     IMG_5559.jpg
10    IMG_5559.jpg
11    IMG_5559.jpg
Name: filename, dtype: object: NewRandomAccessFile failed to Create/Open: dataset/images\9     IMG_5559.jpg
10    IMG_5559.jpg
11    IMG_5559.jpg
Name: filename, dtype: object : The filen

In [17]:
import os
import glob
import pandas as pd
import tensorflow as tf
from lxml import etree
from PIL import Image
from object_detection.utils import dataset_util
import io

# Path to images and annotations
IMAGE_PATH = 'dataset/images'
ANNOTATION_PATH = 'dataset/annotations'

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(os.path.join(path, '*.xml')):
        tree = etree.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def class_text_to_int(row_label):
    if row_label == 'bee':  # Change this to your object class name
        return 1
    else:
        raise ValueError(f"Unknown class label: {row_label}")

def create_tf_example(group, path):
    filename = group['filename'].iloc[0]
    try:
        with tf.io.gfile.GFile(os.path.join(path, filename), 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
    except Exception as e:
        print(f"Error reading image file {filename}: {e}")
        return None

    filename_encoded = filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():
        try:
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
        except ValueError as e:
            print(f"Error processing row for file {filename}: {e}")
            return None

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename_encoded),
        'image/source_id': dataset_util.bytes_feature(filename_encoded),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main():
    writer = tf.io.TFRecordWriter('train.record')
    path = IMAGE_PATH
    examples = xml_to_csv(ANNOTATION_PATH)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        print(f"Processing file: {filename}")
        file_path = os.path.join(path, filename)
        print(f"Path: {file_path}")
        if not os.path.exists(file_path):
            print(f"File {filename} does not exist!")
            continue  # Skip if the file does not exist
        tf_example = create_tf_example(group, path)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
        else:
            print(f"Skipped file {filename} due to read error or unknown class label.")
    writer.close()

# Call the main function directly
if __name__ == '_main_':
    main()